1. 일단 모든 feature에 대해서 neural network 돌려본 결과



In [4]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 로드
data = pd.read_csv("mon.csv")

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Neural Network 모델 정의
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],), activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(95, activation='softmax')  # 95개의 클래스로 분류하기 위한 출력층
])

# 모델 컴파일 (학습률 조정)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# 모델 평가 (accuracy와 f1 score)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# 예측 및 F1 Score 계산
y_pred = np.argmax(model.predict(X_test), axis=1)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.2f}")

# 자세한 분류 보고서 출력
print(classification_report(y_test, y_pred))


Epoch 1/100
416/416 [==============================] - 3s 4ms/step - loss: 4.5970 - accuracy: 0.0232 - val_loss: 4.2033 - val_accuracy: 0.0877
Epoch 2/100
416/416 [==============================] - 2s 4ms/step - loss: 4.1612 - accuracy: 0.0719 - val_loss: 3.8324 - val_accuracy: 0.1449
Epoch 3/100
416/416 [==============================] - 2s 4ms/step - loss: 3.8916 - accuracy: 0.1171 - val_loss: 3.5874 - val_accuracy: 0.1930
Epoch 4/100
416/416 [==============================] - 1s 3ms/step - loss: 3.6642 - accuracy: 0.1444 - val_loss: 3.3962 - val_accuracy: 0.2207
Epoch 5/100
416/416 [==============================] - 1s 3ms/step - loss: 3.4931 - accuracy: 0.1758 - val_loss: 3.2326 - val_accuracy: 0.2565
Epoch 6/100
416/416 [==============================] - 1s 3ms/step - loss: 3.3635 - accuracy: 0.1987 - val_loss: 3.1045 - val_accuracy: 0.2786
Epoch 7/100
416/416 [==============================] - 1s 3ms/step - loss: 3.2396 - accuracy: 0.2165 - val_loss: 2.9874 - val_accuracy: 0.3011